In [ ]:
from manim import *
from scipy.stats import f
import numpy as np
from manim_voiceover import VoiceoverScene
from manim_voiceover.services.gtts import GTTSService



class ANOVAAnimation(VoiceoverScene):
    def construct(self):
        
        
        # Definindo a fala de IA para ter o tempo do vídeo correto para cada animação
        self.set_speech_service(GTTSService(lang="pt"))

        ##########################################
        ####### Criação de todos os elementos
        ##########################################

        # Ajustes de posicionamento
        LEFT_SECTION = LEFT * 2.5  # Movendo tudo para a esquerda
        RIGHT_SECTION = RIGHT * 4  # Movendo tudo mais para a direita

        # Criando o círculo grande que engloba os grupos
        big_circle = Circle(radius=3.2, color=WHITE).shift(LEFT_SECTION)

        # Criando os três grupos com círculos inicialmente maiores
        initial_group_radius = 1.2 
        final_group_radius = 0.35  
        group1 = Circle(radius=initial_group_radius, color=BLUE).shift(LEFT_SECTION + LEFT * 1)
        group2 = Circle(radius=initial_group_radius, color=GREEN).shift(LEFT_SECTION + RIGHT * 1)
        group3 = Circle(radius=initial_group_radius, color=RED).shift(LEFT_SECTION + UP * 0.3)

        # Criando rótulos para os grupos
        label1 = Text("Palestra", color=BLUE).scale(0.5).next_to(group1, DOWN)
        label2 = Text("Workshop", color=GREEN).scale(0.5).next_to(group2, DOWN)
        label3 = Text("Online", color=RED).scale(0.5).next_to(group3, UP)

        # Função para gerar pontos dentro de um círculo específico
        def generate_points(center, radius, color, spread_factor=0.2):
            points = VGroup()
            for _ in range(10):
                while True:
                    x = np.random.uniform(-radius, radius)
                    y = np.random.uniform(-radius, radius)
                    point = np.array([x, y, 0]) + center
                    if np.linalg.norm(point - center) <= radius:
                        break
                points.add(Dot(point=point, color=color))
            return points

        # Gerando pontos dentro dos grupos
        points_group1 = generate_points(group1.get_center(), initial_group_radius - 0.1, BLUE)
        points_group2 = generate_points(group2.get_center(), initial_group_radius - 0.1, GREEN)
        points_group3 = generate_points(group3.get_center(), initial_group_radius - 0.1, RED)

        # Criando a média geral (ponto branco)
        global_mean = Dot(color=WHITE).scale(1.2).shift(LEFT_SECTION)

        # Criando as médias dos grupos
        group1_mean = Dot(color=PURE_BLUE).scale(1.0).move_to(group1.get_center())
        group2_mean = Dot(color=PURE_GREEN).scale(1.0).move_to(group2.get_center())
        group3_mean = Dot(color=PURE_RED).scale(1.0).move_to(group3.get_center())

        # Fórmula do F de Fisher
        formula_F = MathTex("F = \\frac{\\text{Variância entre grupos}}{\\text{Variância dentro dos grupos}}"
        ).scale(0.7).to_edge(UP).shift(RIGHT_SECTION)  # Movido mais para a direita


        formula_F[0][2:23].set_color(ORANGE)
        formula_F[0][24:34].set_color(BLUE)
        formula_F[0][34:40].set_color(GREEN)
        formula_F[0][40:50].set_color(RED)


        # Criando eixo X e Y para o gráfico da distribuição F
        f_distribution = Axes(
            x_range=[0, 6, 1],  # Eixo x com valores inteiros
            y_range=[0, 1, 0.2],
            axis_config={"color": WHITE}
        ).scale(0.7).shift(RIGHT_SECTION + RIGHT * 1.5 + DOWN * 1.2)  # Movido mais para a direita

        v = w = 5
        f_curve = f_distribution.plot(lambda x: f.pdf(x, v, w), color=YELLOW)

        # Adicionando ponto no gráfico para o primeiro momento (x < 2)
        point_initial = Dot(f_distribution.c2p(1, f.pdf(1, v, w)), color=WHITE, radius=0.07)
        area_initial = f_distribution.get_area(f_curve, x_range=[1, 6], color=BLUE, opacity=0.5)

        # Adicionando ponto no gráfico para o segundo momento (x = 4)
        point_final = Dot(f_distribution.c2p(3, f.pdf(3, v, w)), color=WHITE, radius=0.07)
        area_final = f_distribution.get_area(f_curve, x_range=[3, 6], color=BLUE, opacity=0.5)


        # Criando setas sólidas entre a média geral e as médias dos grupos
        arrow_global_to_g1 = always_redraw(lambda: Arrow(global_mean.get_center(), group1_mean.get_center(), color=ORANGE))
        arrow_global_to_g2 = always_redraw(lambda: Arrow(global_mean.get_center(), group2_mean.get_center(), color=ORANGE))
        arrow_global_to_g3 = always_redraw(lambda: Arrow(global_mean.get_center(), group3_mean.get_center(), color=ORANGE))

        # Criando setas tracejadas das médias dos grupos para um ponto dentro do grupo
        arrow_g1_to_point = always_redraw(lambda: DashedLine(group1_mean.get_center(), points_group1[9].get_center(), color=BLUE))
        arrow_g2_to_point = always_redraw(lambda: DashedLine(group2_mean.get_center(), points_group2[9].get_center(), color=GREEN))
        arrow_g3_to_point = always_redraw(lambda: DashedLine(group3_mean.get_center(), points_group3[9].get_center(), color=RED))




        ##########################################
        ####### Iniciando animação
        ##########################################
        with self.voiceover(text="Em uma sala de aula selecionamos aleatóriamente 30 alunos") as tracker:
            self.play(Create(big_circle), Create(points_group1), Create(points_group2), Create(points_group3), run_time=tracker.duration)


        # Adicionando elementos à cena
        #self.play(Create(big_circle))
        #self.play(Create(global_mean), Create(points_group1), Create(points_group2), Create(points_group3))
        #self.wait(2)
        with self.voiceover(text="Em seguida os alunos foram divididos em 3 grupos:") as tracker:
            self.play(Create(group1), Create(group2), Create(group3), run_time=tracker.duration)

        with self.voiceover(text="palestra, workshop e aprendizado online") as tracker:
            self.play((Write(label1), Write(label2), Write(label3)), run_time=tracker.duration)

        with self.voiceover(text="Com isso queremos comparar a eficácia dos 3 métodos de ensino nas notas dos exames dos alunos"):
            self.wait(1)
            
        
        with self.voiceover(text="Para isso precisamos primeiro calcular a média da nota de toda a amostra") as tracker:
            self.play((Create(global_mean)), run_time=tracker.duration)

        with self.voiceover(text="e também a média da nota de cada grupo.") as tracker:
            self.play((Create(group1_mean), Create(group2_mean), Create(group3_mean)), run_time=tracker.duration)    


        # Criando as setas sólidas entre a média geral e as médias dos grupos
        with self.voiceover(text="Em seguida precisamos calcular a variância entre a média global e a média dos grupos") as tracker:
            self.play(Create(arrow_global_to_g1), Create(arrow_global_to_g2), Create(arrow_global_to_g3), run_time = tracker.duration)
        
        # Criando as setas tracejadas das médias dos grupos para um ponto dentro de cada grupo
        with self.voiceover(text="e a variância entre a média e as notas individuais dentro de cada grupo.") as tracker:
            self.play(Create(arrow_g1_to_point), Create(arrow_g2_to_point), Create(arrow_g3_to_point), run_time=tracker.duration)                  
        
                
        # Fórmula ANOVA    
        with self.voiceover(text= "Essa é a base da estatística F da ANOVA, que compara a variância entre os grupos com a variância dentro dos grupos.") as tracker:
            self.play(Write(formula_F), run_time = tracker.duration)

        # Explicando fórmula e visualização das setas intra e entre grupos
        ## Entre grupos

        # 
        with self.voiceover(text= "No exemplo que temos na tela podemos perceber que a variância entre os grupos ") as tracker:
            self.play(Indicate(arrow_global_to_g1, color=DARK_BLUE, scale_factor=1.4), 
                      Indicate(arrow_global_to_g2, color=DARK_BLUE, scale_factor=1.4), 
                      Indicate(arrow_global_to_g3, color=DARK_BLUE, scale_factor=1.4), 
                      run_time=tracker.duration)

        # 
        with self.voiceover(text= "não parece ser tão diferente da variância dentro dos grupos") as tracker:
            self.play(Indicate(arrow_g1_to_point, color=DARK_BLUE, scale_factor=1.4), 
                      Indicate(arrow_g2_to_point, color=DARK_BLUE, scale_factor=1.4), 
                      Indicate(arrow_g3_to_point, color=DARK_BLUE, scale_factor=1.4), 
                      run_time=tracker.duration)
                      
                      
        ## Destaque fórmula
        
        with self.voiceover(text= "Com isso, teremos números bem parecidos tanto na parte de cima") as tracker:
            self.play((Indicate(formula_F[0][2:23], color=DARK_BLUE, scale_factor=1.4)), run_time = tracker.duration)

        with self.voiceover(text= "quanto na parte de baixo da fórmula") as tracker:
            self.play((Indicate(formula_F[0][24:50], color=DARK_BLUE, scale_factor=1.2)), run_time = tracker.duration)

        with self.voiceover(text= "Resultando em um valor de F bem próximo de 1") as tracker:
            self.play((Indicate(formula_F[0][0], color=DARK_BLUE, scale_factor=1.4)), run_time = tracker.duration)

        

        # Adicionando o gráfico

        with self.voiceover(text= "Ao colocar esse valor na distribuição dos possíveis valores de F") as tracker:
            self.play((Create(f_distribution), Create(f_curve), Create(point_initial)), run_time = tracker.duration)

        with self.voiceover(text= "notamos que a probabilidade de encontrar um valor maior que o observado é alta") as tracker:
            self.play((Create(area_initial)), run_time = tracker.duration)

        with self.voiceover(text= "Podemos concluir que os grupos não são diferentes entre si"):
            self.wait(1)

        with self.voiceover(text= "Visualmente podemos perceber isso também"):
            self.wait(1) 

        with self.voiceover(text= "Uma vez que as notas de indivíduos de um grupo, se parecem muito com as notas de indivíduos de outro grupo"):
            self.wait(1)           

        #with self.voiceover(text= "") as tracker:
         #   self.play((Create(area_initial)), run_time = tracker.duration) 

       # with self.voiceover(text= "Essa probabilidade é representada pela área abaixo da curva") as tracker:
        #    self.play((Create(area_initial)), run_time = tracker.duration) 

        #with self.voiceover(text= "a partir do valor de F que calculamos") as tracker:
         #   self.play((Indicate(area_initial, color=DARK_BLUE, scale_factor=1)), run_time = tracker.duration) 
             
            


       # Transição para os grupos se afastando e reduzindo o tamanho
      


        with self.voiceover(text= "Caso a variância entre os grupos fosse alta, ou seja, a média de cada grupo é bem diferente da média geral") as tracker:
            self.play((
            group1.animate.shift(LEFT * 1).scale(0.5),  
            group2.animate.shift(RIGHT * 1).scale(0.5),
            group3.animate.shift(UP * 1).scale(0.5),
            points_group1.animate.shift(LEFT * 0.1),
            points_group2.animate.shift(RIGHT * 0.1),
            points_group3.animate.shift(UP * 0.1),
            group1_mean.animate.shift(LEFT * 1),
            group2_mean.animate.shift(RIGHT * 1),
            group3_mean.animate.shift(UP * 1),
            label1.animate.shift(UL * 0.6 + LEFT * 0.4),
            label2.animate.shift(UR * 0.6 + RIGHT * 0.4),
            label3.animate.shift(UP * 0.5)),
            run_time = tracker.duration)

             # No segundo momento, centralizar os pontos dentro dos círculos menores
        centered_points_group1 = generate_points(group1.get_center(), final_group_radius, BLUE, spread_factor=0.1)
        centered_points_group2 = generate_points(group2.get_center(), final_group_radius, GREEN, spread_factor=0.1)
        centered_points_group3 = generate_points(group3.get_center(), final_group_radius, RED, spread_factor=2)
        
        with self.voiceover(text= "E a variância dentro dos grupos é baixa, indicando que alunos de um mesmo grupo tiram notas parecidas") as tracker:
            self.play((Transform(points_group1, centered_points_group1),
                       Transform(points_group2, centered_points_group2),
                       Transform(points_group3, centered_points_group3)),
                       run_time = tracker.duration)


        
        # Atualizando fórmula do F para valores significativos
        # Transição para o segundo ponto na curva (x = 4)

        with self.voiceover(text= "O valor de F seria maior") as tracker:
            self.play((Transform(point_initial, point_final)), run_time = tracker.duration)

        with self.voiceover(text= "e a probabilidade de encontrar um valor mais extremo, seria menor") as tracker:
            self.play((Transform(area_initial, area_final), Indicate(area_final, color=DARK_BLUE, scale_factor=1)), run_time = tracker.duration)
        
      
        with self.voiceover(text= "Nesse caso, podemos concluir que os grupos são bem diferentes entre si"):
            self.wait(5)

        with self.voiceover(text= "E para quantificar essa diferença precisamos realizar o post-hoc"):
            self.wait(5)

In [1]:
ANOVAAnimation().render()


NameError: name 'ANOVAAnimation' is not defined